# 📊 Phase 1: Simulation Data Exploration

**Purpose**: Explore and visualize the synthetic space tracking dataset

**Author**: Space AI Project  
**Date**: 2026-02-04

---

## Overview

This notebook explores the simulation data generated by Phase 1:
- Ground truth orbital trajectories
- Sensor measurements with realistic noise
- Data quality and coverage analysis
- Visualization of orbits and sensor coverage

In [ ]:
# Setup
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from src.simulation.data_generator import Dataset
from src.utils.coordinates import eci_to_geodetic

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✅ Imports successful")

## 1. Load Dataset

In [ ]:
# Load the quick test dataset
dataset_path = Path('../data/processed/quick_test')
dataset = Dataset.load(dataset_path)

print(f"📦 Dataset loaded from: {dataset_path}")
print(f"\n📊 Statistics:")
print(f"  Objects: {len(dataset.ground_truth['object_id'].unique())}")
print(f"  Ground truth points: {len(dataset.ground_truth)}")
print(f"  Measurements: {len(dataset.measurements)}")
print(f"  Sensors: {len(dataset.measurements['sensor_id'].unique())}")
print(f"  Time span: {(dataset.ground_truth['time'].max() - dataset.ground_truth['time'].min()).total_seconds() / 3600:.2f} hours")

## 2. Summary

This notebook provides comprehensive exploration of Phase 1 simulation data. Run all cells to visualize:
- 3D orbital trajectories
- Altitude profiles
- Measurement coverage
- Noise characteristics

**Next**: Use this data for Phase 2 tracking algorithms!